In [8]:
from google.colab.patches import cv2_imshow

In [9]:
from glob import glob
from skimage import data, io, img_as_ubyte
import numpy as np
import pandas as pd

In [10]:
# to save as tif in google drive
!pip install imagecodecs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [12]:
work_dir = '/content/drive/MyDrive/0_Lymphoma_PPFE_vs_Cryo/'          #you might need to change it in your case, look at your folder structure
man_dir = work_dir + 'images_1_MASKS/'
auto_dir = work_dir + 'images_1_AUTO/alfa/'
save_dir = work_dir + 'images_1_MASK_comparison/'

In [18]:
man_masks = sorted(glob(man_dir + "*_MASK*"))
man_masks.pop(9)

'/content/drive/MyDrive/0_Lymphoma_PPFE_vs_Cryo/images_1_MASKS/1_L32 Cryo Exp 15 Gain 1 Satur 100_MASK.tif'

In [19]:
auto_masks = sorted(glob(auto_dir + "*_MASK*"))

In [20]:
len(man_masks)

31

In [21]:
len(auto_masks)

31

In [22]:
df = pd.DataFrame(data = {'img':[], 'dice': []})
for i in range(len(man_masks)):
  img = man_masks[i]
  img2 = auto_masks[i]
  man_0 = io.imread(img)
  man_0[man_0 == 1] = 0
  man_0 = 255 - man_0
  auto_0 = io.imread(img2)
  #DICE
  im1 = man_0.astype(bool)
  im2 = auto_0.astype(bool)
  intersection = np.logical_and(im1, im2)
  dice = 2. * intersection.sum() / (im1.sum() + im2.sum())
  #PLOT
  man_0[man_0 == 255] = 200
  man_0[man_0 == 0] = 100
  auto_0[auto_0 == 255] = 50
  auto_0[auto_0 == 0] = 25
  comparison = man_0 - auto_0
  # cv2_imshow(comparison)
  maskname = save_dir+img.split('/')[-1][:-4]+"_COMPARISON_DICE_"+str(round(dice, 3))+".tif"
  io.imsave(maskname, comparison)
  df.loc[df.shape[0]] = [img.split('/')[-1][:-4], dice]
df.to_excel(save_dir+'dice_coef.xlsx', index=False)